In [1]:
from google.colab import files
upload = files.upload()

Saving celulares_limpios_actualizado.json to celulares_limpios_actualizado.json


In [2]:
from google.colab import files
upload = files.upload()

Saving intenstss.json to intenstss.json


In [3]:
from google.colab import files
upload = files.upload()

Saving preguntas_prueba2.json to preguntas_prueba2.json


In [4]:
pip install transformers torch sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [5]:
!git clone https://github.com/MadisonAcosta/PyTutor.git


Cloning into 'PyTutor'...
remote: Enumerating objects: 244, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 244 (delta 63), reused 117 (delta 38), pack-reused 95 (from 2)
Receiving objects: 100% (244/244), 42.94 MiB | 9.76 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [15]:
import json
import torch
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Cargar modelo BERT en español
modelo_nombre = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
tokenizer = AutoTokenizer.from_pretrained(modelo_nombre)
modelo = AutoModel.from_pretrained(modelo_nombre)

# Cargar los datos de intenciones y celulares
with open("intenstss.json", "r", encoding="utf-8") as file:
    intenciones = json.load(file)

with open("celulares_limpios_actualizado.json", "r", encoding="utf-8") as file:
    celulares = json.load(file)

# Cargar las preguntas de prueba con respuestas esperadas
with open("preguntas_prueba2.json", "r", encoding="utf-8") as file:
    pruebas = json.load(file)

# Función para obtener embeddings de una oración con BERT
def obtener_embedding(texto):
    inputs = tokenizer(texto, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = modelo(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Función para obtener la respuesta del chatbot usando BERT
def chatbot_responde(pregunta):
    emb_pregunta = obtener_embedding(pregunta)
    mejor_respuesta = None
    mejor_similitud = 0.0

    for intent in intenciones["intents"]:
        for pattern in intent["patterns"]:
            emb_pattern = obtener_embedding(pattern)
            similitud = cosine_similarity(emb_pregunta, emb_pattern)[0][0]

            if similitud > mejor_similitud:
                mejor_similitud = similitud
                mejor_respuesta = intent["responses"]

    return mejor_respuesta if mejor_respuesta else "Lo siento, no entiendo la pregunta."

# Evaluación del chatbot
aciertos = 0
total_pruebas = len(pruebas)
umbral_similitud = 0.50  #ajuste adaptado

with open("preguntas_prueba2.json", "r", encoding="utf-8") as file:
    pruebas = json.load(file)#verificación de datos
    if isinstance(pruebas, list) and isinstance(pruebas[0], str):#evaluar si existen listas
        pruebas = [{"pregunta": p, "respuesta_esperada": ""} for p in pruebas]
    elif not isinstance(pruebas, list):
        print("Error: El json no tiene formato lista")


    for prueba in pruebas:  #identificar la respuesta esperada
        pregunta = prueba["pregunta"]
        respuesta_esperada = prueba.get("respuesta_esperada", "")

        respuesta_bot = chatbot_responde(pregunta)

# Lista para almacenar las similitudes
similitudes = []

for prueba in pruebas:  # Identificar la respuesta esperada
    pregunta = prueba["pregunta"]
    respuesta_esperada = prueba.get("respuesta_esperada", "")

    respuesta_bot = chatbot_responde(pregunta)

    # Calcular similitud entre la respuesta esperada y la del bot
    emb_bot = obtener_embedding(respuesta_bot)
    emb_esperada = obtener_embedding(respuesta_esperada)
    similitud = cosine_similarity(emb_bot, emb_esperada)[0][0]

    # Guardar la similitud en la lista
    similitudes.append(similitud)

    print(f"\nPregunta: {pregunta}")
    print(f"Esperada: {respuesta_esperada}")
    print(f"Bot: {respuesta_bot}")
    print(f"Similitud: {similitud:.2f}")

    if similitud >= umbral_similitud:
        aciertos += 1

# Cálculo de eficiencia
eficiencia = (aciertos / total_pruebas) * 100


# Cálculo del promedio de similitud
promedio_similitud = sum(similitudes) / len(similitudes)

print(f"\nLa Eficiencia del chatbot: {eficiencia:.2f}%")
print(f"Promedio de Similitud: {promedio_similitud:.2f}")



Pregunta: ¿Cuál es el mejor celular?
Esperada: El mejor celular es el iPhone 16 Pro Max.
Bot: ['Si buscas un celular para juegos exigentes, los ASUS ROG Phone o los iPhone 16 Pro Max son top.', '¿Te gustan juegos como PUBG o COD? Te sugiero uno con Snapdragon 8 Gen 2 o A17 Pro.', 'Mira los modelos con pantallas de 120Hz o más, ideales para gaming fluido.', 'Algunos celulares traen triggers físicos para mejorar la jugabilidad. ¿Te muestro algunos?', '¿Qué presupuesto manejas para un celular gamer? Puedo darte opciones de gama alta o media.']
Similitud: 0.70

Pregunta: ¿Qué celular tiene mejor cámara?
Esperada: El celular con mejor cámara es el Samsung Galaxy S24 Ultra.
Bot: ['Si te encanta la fotografía, los Google Pixel 8 Pro y los iPhone 16 Pro Max son geniales.', '¿Quieres buenos selfies? Te recomiendo el iPhone 16 o el Pixel 7a con excelentes cámaras frontales.', 'Para video, muchos modelos ya graban en 4K e incluso 8K. ¿Qué te gustaría grabar?', '¿Necesitas zoom óptico? Los Galaxy

**Interpretación**
*   El modelo segun eficiencia arriba de un humbral del 50% se posee un 46.15% resultante, esto desarrollado bajo el enfoque de precision

*   generando un promedio de similitudes se poseen un 49% en relacion a lo que hipoteticamente se espera es respuesta a lo que el bot genera

*   las preguntas y respuestas esperadas son producto de hipotesis



